In [109]:
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns  
from linares_plot import *

In [ ]:
frames=[]

Trucos=['Truco 1', 'Truco 2', 'Truco 3', 'Truco 4', 'Truco 5', 'Truco 6', 'Truco 7', 'Truco 8' ]

for T in Trucos:
    Df=pd.read_excel(os.getcwd() + '\\ANOTACIONES_.xlsx', sheet_name= T )
    subj_idx = Df.columns[2:]
    #   
    for s in subj_idx:
        df_x = Df[['Tiempo_s', 'Tiempo', s]]
        df_x.columns=['time_s', 'time_min_sec', 'labels']
        df_x['subject']=s
        df_x['trick']=int(T.split(' ')[1])
        frames.append(df_x)

df = pd.concat(frames, ignore_index=True)

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [ ]:
df_ = df.loc[~pd.isna(df['labels'])]
df_.head()

In [107]:
n_actions = len(df_.labels.iloc[0].split(','))

items=[]

for i in range(len(df_)):
    print(i)
    n_actions = len(df_.labels.iloc[i].split(','))
    if n_actions==1:
        items.append([df_.time_s.iloc[i], df_.time_min_sec.iloc[i], 
                      df_.labels.iloc[i], df_.subject.iloc[i],  df_.trick.iloc[i]  ])
    elif n_actions>1:
        actions = df_.labels.iloc[i].split(',')
        for a in range(0, n_actions):
            items.append([df_.time_s.iloc[i], df_.time_min_sec.iloc[i], 
                          actions[a], df_.subject.iloc[i],  df_.trick.iloc[i]  ])
            
            
df_actions = pd.DataFrame(actions)    
df_actions.columns = df_columns

df_actions

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

AttributeError: 'int' object has no attribute 'split'

In [108]:
df_.iloc[1333]

time_s               340
time_min_sec    00:05:40
labels                 0
subject               33
trick                  1
Name: 6250, dtype: object

In [28]:
#df['labels']=df['labels'].replace(df.labels.iloc[0], 0) ##in case I have to get rid of the nans and put 0s


In [11]:
Df=pd.read_excel(os.getcwd() + '\\ANOTACIONES_.xlsx', sheet_name='Truco 1')
Df

,Tiempo_s,Tiempo,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,50
0,0,00:00:00,NaN,AA,A,A,A+,A,A,"S,A",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,00:06:00,NaN,B,NaN,NaN,NaN,NaN,P,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,00:08:00,Se,NaN,NaN,NaN,NaN,B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10,00:10:00,B,NaN,NaN,O,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12,00:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,14,00:14:00,NaN,NaN,NaN,NaN,NaN,P,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,16,00:16:00,NaN,NaN,NaN,NaN,NaN,NaN,C,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,18,00:18:00,NaN,NaN,NaN,B,NaN,NaN,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
Df.Tiempo

0      00:00:00
1      00:02:00
2      00:04:00
3      00:06:00
4      00:08:00
5      00:10:00
6      00:12:00
7      00:14:00
8      00:16:00
9      00:18:00
10     00:20:00
11     00:22:00
12     00:24:00
13     00:26:00
14     00:28:00
15     00:30:00
16     00:32:00
17     00:34:00
18     00:36:00
19     00:38:00
20     00:40:00
21     00:42:00
22     00:44:00
23     00:46:00
24     00:48:00
25     00:50:00
26     00:52:00
27     00:54:00
28     00:56:00
29     00:58:00
         ...   
266         NaN
267         NaN
268         NaN
269         NaN
270         NaN
271         NaN
272         NaN
273         NaN
274         NaN
275         NaN
276         NaN
277         NaN
278         NaN
279         NaN
280         NaN
281    09:22:00
282    09:24:00
283    09:26:00
284    09:28:00
285    09:30:00
286    09:32:00
287    09:34:00
288    09:36:00
289    09:38:00
290    09:40:00
291    09:42:00
292    09:44:00
293    09:46:00
294    09:48:00
295    09:50:00
Name: Tiempo, Length: 29